* The temporal order is scrambled in the test data making TS useless there.
* Still, just for learning / realism, we can still do it in the training data! 
* Let's add pseudo dates, and aggregate features on column subsets. Finally i'll run a model to predict the target!

In [56]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from datetime import datetime
from scipy.special import logsumexp

from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import mean_squared_error, classification_report

In [57]:
train = pd.read_csv("caltech-cs155-2020/train.csv")
test = pd.read_csv("caltech-cs155-2020/test.csv")
df = pd.concat([train,test],sort=False)
print(df.shape)
print(df.columns)
df.tail()

(784239, 28)
Index(['id', 'last_price', 'mid', 'opened_position_qty ',
       'closed_position_qty', 'transacted_qty', 'd_open_interest', 'bid1',
       'bid2', 'bid3', 'bid4', 'bid5', 'ask1', 'ask2', 'ask3', 'ask4', 'ask5',
       'bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol', 'ask1vol',
       'ask2vol', 'ask3vol', 'ask4vol', 'ask5vol', 'y'],
      dtype='object')


,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
191854,784234,5150.6,5150.8,NaN,NaN,0.0,0,5150.6,5150.4,5150.2,...,5,3,4,2,1,3,1,3,2,NaN
191855,784235,5122.6,5122.7,1.0,2.0,3.0,-1,5122.6,5122.4,5122.2,...,1,5,1,20,16,17,4,1,16,NaN
191856,784236,5192.8,5192.3,NaN,NaN,0.0,0,5192.0,5191.8,5191.2,...,2,1,1,1,1,2,1,2,1,NaN
191857,784237,5152.0,5151.9,4.0,2.0,6.0,2,5151.8,5151.6,5151.4,...,5,6,8,37,7,1,2,1,1,NaN
191858,784238,5200.0,5200.3,2.0,0.0,2.0,2,5200.0,5199.0,5198.8,...,5,1,2,1,1,1,4,1,3,NaN


In [58]:
train.head()

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
0,0,3842.4,3842.6,NaN,NaN,103.0,0,3842.4,3842.0,3841.8,...,1,6,14,6,6,1,1,10,2,1
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,3843.0,3842.8,3842.4,...,6,11,1,6,1,4,4,1,13,0
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3843.8,3843.6,3843.2,...,1,4,21,12,1,16,10,4,9,0
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,3843.0,3842.8,3842.4,...,13,12,2,4,2,7,1,2,11,1
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,3842.8,3842.4,3842.0,...,12,2,2,4,1,3,1,11,15,1


In [59]:
test.head()

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid1vol,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol
0,592380,5178.4,5178.3,1.0,11.0,12.0,-10,5178.2,5178.0,5177.8,...,3,16,3,1,1,1,4,1,5,2
1,592381,5133.0,5132.9,1.0,0.0,1.0,1,5132.8,5132.6,5132.4,...,1,2,2,1,10,4,8,1,1,1
2,592382,5177.4,5178.0,2.0,0.0,2.0,2,5177.2,5176.8,5176.6,...,8,18,1,1,6,7,4,1,2,5
3,592383,5093.6,5093.9,0.0,2.0,2.0,-1,5093.2,5093.0,5092.8,...,8,5,4,3,1,1,1,11,6,2
4,592384,5189.2,5189.2,NaN,NaN,0.0,0,5188.8,5188.6,5188.4,...,1,5,1,3,3,3,2,1,3,4


In [60]:
## y is binary.
display(train["y"].describe())

count    592380.000000
mean          0.356958
std           0.479103
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: y, dtype: float64

In [61]:
bid_cols = ['bid1','bid2', 'bid3', 'bid4', 'bid5']
bid_vol_cols = ['bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol']
ask_cols = ['ask1', 'ask2', 'ask3', 'ask4', 'ask5',]
ask_vol_cols = ['ask1vol','ask2vol', 'ask3vol', 'ask4vol', 'ask5vol']

group_cols = {"bid_cols":bid_cols,"bid_vol_cols":bid_vol_cols,"ask_cols":ask_cols,"ask_vol_cols":ask_vol_cols}

* Additional features could include: rank, which bid number is the max/min, etc' 
* features between the aggregated features (e.g. max bid div max ask..)

In [62]:
for group in group_cols.keys():
    print(group)
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    df[f"{group}_min"] = df[group_cols[group]].min(axis=1)
    df[f"{group}_spread"] = df[f"{group}_max"].div(df[f"{group}_min"])
#     df[f"{group}_logsumexp"] = df[grou/_cols[group]].apply(logsumexp)
    
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    
df["last_price_div__mid"] = df["last_price"].div(df["mid"])

bid_cols
bid_vol_cols
ask_cols
ask_vol_cols


In [63]:
df["date"] = pd.to_datetime("1.1.2019")
df["date"] = df["date"] + pd.to_timedelta(df["id"]/2,unit="s") # 500 ms per row

df["date"].describe()

count                         784239
unique                        784239
top       2019-01-02 14:21:54.500000
freq                               1
first            2019-01-01 00:00:00
last             2019-01-05 12:55:19
Name: date, dtype: object

# Split back into train and test, and build model

In [64]:
train = df.loc[~df.y.isna()]
print(f"train shape {train.shape[0]}")
test = df.loc[df.y.isna()]
print(f"test shape {test.shape[0]}")

train shape 592380
test shape 191859


In [65]:
train_no_nan = train.fillna(train.mean())
test_no_nan = test.fillna(test.mean())

In [66]:
train_no_nan.isna()

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [67]:
train.drop(["id"],axis=1).to_csv("train_hft.csv.gz",index=False,compression="gzip")
test.to_csv("test_hft_nodates.csv.gz",index=False,compression="gzip")

In [68]:
# we don't know if the test set has a temporal split, so we'll just try a random split for now
X = train_no_nan.drop(["id","date","y"],axis=1)
y = train_no_nan["y"]

In [69]:
print(X.shape)
print(np.isfinite(X).all())

(592380, 39)
last_price              True
mid                     True
opened_position_qty     True
closed_position_qty     True
transacted_qty          True
d_open_interest         True
bid1                    True
bid2                    True
bid3                    True
bid4                    True
bid5                    True
ask1                    True
ask2                    True
ask3                    True
ask4                    True
ask5                    True
bid1vol                 True
bid2vol                 True
bid3vol                 True
bid4vol                 True
bid5vol                 True
ask1vol                 True
ask2vol                 True
ask3vol                 True
ask4vol                 True
ask5vol                 True
bid_cols_max            True
bid_cols_min            True
bid_cols_spread         True
bid_vol_cols_max        True
bid_vol_cols_min        True
bid_vol_cols_spread     True
ask_cols_max            True
ask_cols_min            True
a

In [70]:
#model = CatBoostClassifier()
from sklearn.ensemble import RandomForestClassifier
# from randomforestorder import RandomForestOrder
# model = RandomForestOrder()
model = RandomForestClassifier(n_estimators=100,max_depth=10)
model.fit(X,y)

C:\Users\George\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [73]:

from sklearn.metrics import log_loss

y_pred = model.predict(train_no_nan.drop(["id","date","y"],axis=1))
print(log_loss(y,y_pred))



0.8106181848096996
0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
5         0.0
6         0.0
7         0.0
8         1.0
9         0.0
10        0.0
11        0.0
12        0.0
13        0.0
14        0.0
15        0.0
16        0.0
17        0.0
18        0.0
19        0.0
20        0.0
21        0.0
22        0.0
23        0.0
24        0.0
25        0.0
26        0.0
27        1.0
28        0.0
29        0.0
         ... 
592350    0.0
592351    0.0
592352    0.0
592353    0.0
592354    0.0
592355    0.0
592356    0.0
592357    0.0
592358    0.0
592359    0.0
592360    0.0
592361    0.0
592362    0.0
592363    0.0
592364    0.0
592365    0.0
592366    0.0
592367    0.0
592368    0.0
592369    0.0
592370    0.0
592371    0.0
592372    0.0
592373    0.0
592374    0.0
592375    0.0
592376    0.0
592377    0.0
592378    0.0
592379    0.0
Name: y, Length: 592380, dtype: float64


In [75]:

print(np.sum(np.abs(y-y_pred))/y.shape[0])

0.02346973226645059


In [76]:
y_prob = model.predict_proba(train_no_nan.drop(["id","date","y"],axis=1))

In [84]:

print(y_prob[:,1])
print(np.sum(np.abs(y-y_prob[:,1]))/y.shape[0])
print(log_loss(y,y_prob[:,1]))

[0.7 0.4 0.2 ... 0.  0.  0. ]
0.15246375129366627
0.18014424102324603


In [ ]:
import get_average_val_err
import importlib
importlib.reload(get_average_val_err)
print(get_average_val_err.get_val_err(5,train_no_nan,model))

592380
Fold  1  of  5  test indices: [     0      1      2 ... 118473 118474 118475]
118476
        last_price     mid  opened_position_qty   closed_position_qty  \
118476      3885.4  3885.5              1.400583             1.964277   
118477      3885.4  3885.3              1.400583             1.964277   
118478      3885.8  3885.5              2.000000             2.000000   
118479      3885.8  3885.5              1.400583             1.964277   
118480      3885.8  3885.6              1.000000             1.000000   

        transacted_qty  d_open_interest    bid1    bid2    bid3    bid4  ...  \
118476             0.0                0  3885.2  3885.0  3884.8  3884.6  ...   
118477             0.0                0  3885.2  3885.0  3884.8  3884.6  ...   
118478             4.0                0  3885.2  3885.0  3884.8  3884.6  ...   
118479             0.0                0  3885.4  3885.2  3885.0  3884.8  ...   
118480             2.0                0  3885.4  3885.2  3885.0  3884

C:\Users\George\Desktop\CS155_kaggle\actual\TeamAPlus\get_average_val_err.py:30: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)


Fold  2  of  5  test indices: [118476 118477 118478 ... 236949 236950 236951]
118476
   last_price     mid  opened_position_qty   closed_position_qty  \
0      3842.4  3842.6              1.400583             1.964277   
1      3842.8  3843.4              6.000000            49.000000   
2      3844.0  3844.3              7.000000            77.000000   
3      3843.8  3843.4              3.000000            34.000000   
4      3843.2  3843.1              3.000000            38.000000   

   transacted_qty  d_open_interest    bid1    bid2    bid3    bid4  ...  \
0           103.0                0  3842.4  3842.0  3841.8  3841.0  ...   
1            55.0              -43  3843.0  3842.8  3842.4  3842.0  ...   
2            84.0              -69  3843.8  3843.6  3843.2  3843.0  ...   
3            37.0              -30  3843.0  3842.8  3842.4  3842.0  ...   
4            41.0              -35  3842.8  3842.4  3842.0  3841.8  ...   

   bid_vol_cols_max  bid_vol_cols_min  bid_vol_cols_spr

C:\Users\George\Desktop\CS155_kaggle\actual\TeamAPlus\get_average_val_err.py:30: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)


Fold  3  of  5  test indices: [236952 236953 236954 ... 355425 355426 355427]
118476
   last_price     mid  opened_position_qty   closed_position_qty  \
0      3842.4  3842.6              1.400583             1.964277   
1      3842.8  3843.4              6.000000            49.000000   
2      3844.0  3844.3              7.000000            77.000000   
3      3843.8  3843.4              3.000000            34.000000   
4      3843.2  3843.1              3.000000            38.000000   

   transacted_qty  d_open_interest    bid1    bid2    bid3    bid4  ...  \
0           103.0                0  3842.4  3842.0  3841.8  3841.0  ...   
1            55.0              -43  3843.0  3842.8  3842.4  3842.0  ...   
2            84.0              -69  3843.8  3843.6  3843.2  3843.0  ...   
3            37.0              -30  3843.0  3842.8  3842.4  3842.0  ...   
4            41.0              -35  3842.8  3842.4  3842.0  3841.8  ...   

   bid_vol_cols_max  bid_vol_cols_min  bid_vol_cols_spr

C:\Users\George\Desktop\CS155_kaggle\actual\TeamAPlus\get_average_val_err.py:30: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)


## export predictions

In [ ]:
# test["Predicted"] = model.predict(test.drop(["id","date","y"],axis=1),prediction_type='Probability')[:,1]
test["Predicted"] = model.predict(test.drop(["id","date","y"],axis=1),prediction_type='Probability')[:,1]
test[["id","Predicted"]].to_csv("submission.csv",index=False)